#Variant Filtration Pipeline for *NBEAL2* project

##Fastq files

All generated fastq files have been deposited to the [NCBI Sequence Read Archive](http://www.ncbi.nlm.nih.gov/sra) (project accession number \#PRJNA296481)

\#PRJNA296481 encompasses 32 whole exome sequencing experiments from 32 different mice representing 8 ENU suppressor lines: *MF5L*1, *MF5L*5, *MF5L*6, *MF5L*8, *MF5L*9, *MF5L*11, *MF5L*12, *MF5L*16.

Current pipeline calls variants jointly from all the 32 sequencing experiments and then focuses on 4 mice from line *MF5L*6: \#SRR2473222, \#SRR2473223, \#SRR2473224, \#SRR2473338


##Overview of variant calling pipeline

Reference genome: Mus_musculus GRCm38 release 73, downloaded from Ensemble

**Used software**: 
* bwa (version 0.7.5a-r405)
* picard-tools (version 1.105)
* GenomeAnalysisTK (version 2.6.5)

**Steps to obtain the .vcf files:**
1. bwa aln -q 15
 bwa sampe
2. picard-tools/SortSam.jar SORT_ORDER=coordinate VALIDATION_STRINGENCY=SILENT CREATE_INDEX=true
3. picard-tools/MarkDuplicates.jar VALIDATION_STRINGENCY=SILENT REMOVE_DUPLICATES=true ASSUME_SORTED=true CREATE_INDEX=true
4. GenomeAnalysisTK/GenomeAnalysisTK.jar -T RealignerTargetCreator
5. GenomeAnalysisTK/GenomeAnalysisTK.jar -T IndelRealigner
6. picard-tools/FixMateInformation.jar SORT_ORDER=coordinate VALIDATION_STRINGENCY=SILENT CREATE_INDEX=true
7. GenomeAnalysisTK/GenomeAnalysisTK.jar -T HaplotypeCaller -stand_call_conf 50.0 stand_emit_conf
8. GenomeAnalysisTK/GenomeAnalysisTK.jar -T VariantAnnotator -A VariantType
9. GenomeAnalysisTK/GenomeAnalysisTK.jar -T SelectVariants -selectType SNP 
10. GenomeAnalysisTK/GenomeAnalysisTK.jar -T SelectVariants -selectType INDEL
11. GenomeAnalysisTK/GenomeAnalysisTK.jar -T VariantFiltration --variant SNP.vcf --filterExpression 'QD < 2.0 || MQ < 40.0 || FS > 60.0 || HaplotypeScore > 13.0 || MQRankSum < -12.5 || ReadPosRankSum < -8.0' --filterName 'FAIL'
12. GenomeAnalysisTK/GenomeAnalysisTK.jar -T VariantFiltration --variant INDEL.vcf --filterExpression "QD < 2.0 || FS > 200.0 || ReadPosRankSum < -20.0 || InbreedingCoeff < -0.8" --filterName FAILED

_End result is two vcf files: SNP.vcf and INDEL.vcf_

##Annotation of called SNP.vcf and INDEL.vcf

Details on Annovar version and reference sequence:

1. $LastChangedDate: 2012-05-25
2. annotate_variation.pl --buildver mm10 --downdb seq mousedb/mm10_seq
_Downloading annotation database ftp://hgdownload.cse.ucsc.edu/goldenPath/mm10/bigZips/chromFa.tar.gz ... OK_

In [ ]:
#UNIX commands:
    
grep '#CHROM' SNP.vcf > header.txt
cat SNP.vcf INDEL.vcf | sort -k1n -k2n | grep -v '#' > merged.vcf

convert2annovar.pl merged.vcf -format vcf4old -includeinfo > merged
annotate_variation.pl merged --buildver mm10 /database/annovar/mousedb/

##Preparation of files for filtration

###Merge and reformat Annovar output files back into one .vcf

Input files are Annovar output files 'merged.exonic_variant_function' and 'merged.variant_function'

In [2]:
#import all necessary packages
import sys
import numpy as np
import re
import vcf
import subprocess
from __future__ import print_function

In [3]:
def reformatAnnovar(FileName,Header):
    '''reformatAnnovar will merge .exonic_variant_function and .variant_function
    back into one vcf file while adding Annovar Annotation into the INFO column'''
    
    input1=open(FileName+'.variant_function','r')
    input2=open(FileName+'.exonic_variant_function','r')
    outfile=open(FileName+'_annot.vcf','w')
    header=open(Header,'r').readline().rstrip()
    rows=[]
    exonic=dict()
    
    print(header,file=outfile)
    
    for line in input2:
        line=line.strip()
        row=line.split('\t')
        row[15]=row[15]+';AnnovarType='+'_'.join(row[1].split())+';AnnovarGene='+'_'.join(row[2].split())
        new_line=row[8:len(row)]        
        new_line='\t'.join(new_line)
        exonic[row[3]+'_'+row[4]]=new_line

    for line in input1:
        line=line.strip()
        row=line.split('\t')
        
        if row[2]+'_'+row[3] in exonic:
            print(exonic[row[2]+'_'+row[3]],file=outfile)
            continue
        else:
            row[14]=row[14]+';AnnovarType='+'_'.join(row[0].split())+';AnnovarGene='+'_'.join(row[1].split())
            new_line=row[7:len(row)]
            print('\t'.join(new_line),file=outfile)
            continue

Here, I merge Annovar annotation files of the original 'merged.vcf' file into **'merged_annot.vcf'** file. 'merged_annot.vcf' file will be used for the downstream filtering.

In [5]:
reformatAnnovar('merged','header.txt')

###Generating a list of known strain specific variation

Before filtering, I am creating a dictionary of all known C57BL/6J and 129/SvImJ SNPs necessary for one of the filtering steps! The .vcf files for these SNPs and INDELs (GRCm38) are downloaded from the [Mouse Genomes Project](http://www.sanger.ac.uk/resources/mouse/genomes/).

In [2]:
def VariantDictionaryFromVcf(VCFfile):
    '''This Function Takes the variants in a given VCF file
    and returns them in a dictionary formatted as chr_pos as keys'''
    input=open(VCFfile)
    variants=dict()
    for line in input:
        if not line.startswith('#'):
            row=line.rstrip().split('\t')
            variants[row[0]+'_'+row[1]]=0
    
    return variants 

In [6]:
IMJ=VariantDictionaryFromVcf('129_mm10_variants.vcf')

## Variant Filtration Pipeline

###FILTER 1
from 'merged_annot.vcf' to 'merged_annot_filter1.vcf'

The goal is to extract heterozygous variants present in the 4 mice from *MF5L*6 suppressor line.

|MouseID|SRA Accession|Name in VCF|
|-|-|-|
|31420|\#SRR2473222|214_461_27|
|33507|\#SRR2473223|214_461_28|
|55574|\#SRR2473224|214_461_29|
|33877|\#SRR2473338|214_461_46|

_Count of all variants in file and of variants that passed the filter are printed to stdout_

In [7]:
vcfFile = vcf.Reader( open('merged_annot.vcf','r') )
outfile = vcf.Writer(open('merged_annot_filter1.vcf', 'w'), vcfFile)

miceIDs=['214_461_27','214_461_28','214_461_29','214_461_46']

#filters statistics
counter=0
passed=0

for record in vcfFile:  
    counter+=1    
    #count how many MF5L6 mice share the called variant
    count_hets=0
    for i in miceIDs:
        if not record.genotype(i)==None and not record.genotype(i)['GT']==None:
            if record.genotype(i)['GT']=='0/1':
                count_hets+=1
                
    #remove variants where none of the MF5L6 mice is heterozygous
    if count_hets==0:
        continue

    passed+=1
    outfile.write_record(record)
    
#print filters statistics    
print('all','passed',sep='\t')   
print(counter,passed,sep='\t')
outfile.close()

all	passed
7012855	1785832


###FILTER 2
from 'merged_annot_filter1.vcf' to 'merged_annot_filter2.vcf'

This step will apply multiple quality filters to identify likely ENU induced mutations in *MF5L*6 mice. In order to pass the filters, the variant has to be:
- heterozygous only in *MF5L*6 mice compared to other sequenced in-house mice
- called in at least 50% of the mice (17) to be able to assess uniqueness
- have at least a read debth of 6 and an allele ratio between 1/2 and 2 for reads DB\<=10 and between 1/3 and 3 for DB\>10
- not present in the list of known C57BL/6J and 129/SvImJ SNPs

_Count of all variants entering this step, of variants removed by each filter, and of variants that passed all filters are printed to stdout_

In [8]:
vcfFile = vcf.Reader( open('merged_annot_filter1.vcf','r') )
outfile = vcf.Writer(open('merged_annot_filter2.vcf', 'w'), vcfFile)

miceIDs=['214_461_27','214_461_28','214_461_29','214_461_46']

#filters statistics
counter=0
(f1,f2,f3,f4,passed)=(0,0,0,0,0)

for record in vcfFile:  
    counter+=1    
    
    #count how many MF5L6 mice share the called heterozygous variant
    count_hets=0
    for i in miceIDs:
        if not record.genotype(i)==None and not record.genotype(i)['GT']==None:
            if record.genotype(i)['GT']=='0/1':
                count_hets+=1
                
    #remove variants where other mice shared the same variant allele as het or alt-hom    
    if record.num_het>count_hets or record.num_hom_alt>0:
        f1+=1
        continue
        
    #remove variants where >50% of the mice (17 mice) have no genotype called     
    if record.num_unknown>=17:
        f2+=1
        continue 

    #called variant should have read depth of 6 at least in one of 4 mice
    #with allele ratios between 1/2 and 2 or 1/3 and 3:
    count_DP=0
    for i in miceIDs:
        if not record.genotype(i)==None and not record.genotype(i)['DP']==None:
            if int(record.genotype(i)['DP'])>=6:
                ref=float(record.genotype(i)['AD'][0])
                alt=float(record.genotype(i)['AD'][1])
                dev=alt/ref
                if alt==0 or ref==0:
                    continue 
                if int(record.genotype(i)['DP'])<=10:
                    if dev<0.5 or dev>2:
                        continue
                if int(record.genotype(i)['DP'])>10:
                    if dev<0.333 or dev>3:
                        continue
                count_DP+=1
                
    if count_DP==0:
        f3+=1
        continue
    
    #remove all variants overlapping with known C57BL/6J and 129S1/SvIMJ SNPs 
    if str(record.CHROM)+'_'+str(record.POS) in IMJ:
        f4+=1
        continue
        
    passed+=1
    outfile.write_record(record)
#print filters statistics   
print('all','shared','<50%GT','badDP','129var','passed',sep='\t')   
print(counter,f1,f2,f3,f4,passed,sep='\t')
outfile.close()

all	shared	<50%GT	badDP	129var	passed
1785832	1681696	46669	51224	6	6237


###FILTER 3
from 'merged_annot_filter2.vcf' to 'merged_annot_filter3.vcf'

This filter will remove variants that are <=200bp apart. Closely called variants are mostly not of ENU origin but false positives do to misalignment

In [3]:
def RemoveVariantsWithin200bp(FileName,OutFile):
    '''Removes variants that are closer than 200 bp to each other'''
    
    input=open(FileName)
    outfile=open(OutFile,'w')
    
    header=input.readline().rstrip()
    print(header,file=outfile)

    chr=''
    pos=0
    removed_pos=0
    rows=[]
    counter=0
    passed=0

    for line in input:
        counter+=1
        line=line.rstrip()
        row=line.split('\t')
        if not row[0]==chr:
            chr=row[0]
            pos=int(row[1])
            rows.append(line)
            removed_pos=0
            continue
        else:
            if int(row[1])-removed_pos <= 200:
                    removed_pos=int(row[1])
                    continue
            else:
                if int(row[1])-pos <= 200:
                    rows.pop()
                    removed_pos=int(row[1])
                    continue
                else:
                    pos=int(row[1])
                    rows.append(line)
                    continue

    for line in rows:
        passed+=1
        print(line,file=outfile)

    #for stats, this is the number of all and discarded variants
    print('all','passed',sep='\t')   
    print(counter,passed,sep='\t')
    outfile.close()

In [4]:
RemoveVariantsWithin200bp('merged_annot_filter2.vcf','merged_annot_filter3.vcf')

all	passed
6237	5946


###FILTER 4
from 'merged_annot_filter3.vcf' to 'merged_annot_filter4.vcf'

This filter removes variants present in only 1 out of 4 *MF5L6* mice

In [5]:
vcfFile = vcf.Reader( open('merged_annot_filter3.vcf','r') )
outfile = vcf.Writer(open('merged_annot_filter4.vcf', 'w'), vcfFile)

miceIDs=['214_461_27','214_461_28','214_461_29','214_461_46']

#filters statistics
counter=0
passed=0

for record in vcfFile:  
    counter+=1    
    
    #count how many MF5L6 mice share the called variant
    count_hets=0
    for i in miceIDs:
        if not record.genotype(i)==None and not record.genotype(i)['GT']==None:
            if record.genotype(i)['GT']=='0/1':
                count_hets+=1
                
    #remove variants present in only 1 mouse:    
    if count_hets<2:
        continue
    else:
        passed+=1
        outfile.write_record(record)
        
#print filters statistics    
print('all','passed',sep='\t')   
print(counter,passed,sep='\t')
outfile.close()

all	passed
5946	94


##Summary statistics

Pull out summary statistics on variant types from three different stages of filtering (for Table 1 in *NBEAL2* manuscript):
- all variants in *MF5L*6 ('merged_annot_filter1.vcf')
- unique heterozygous variants in *MF5L*6 ('merged_annot_filter3.vcf')
- unique heterozygous variants shared by more than 1 *MF5L*6 mouse ('merged_annot_filter4.vcf')

In [6]:
#get a sense of what type of VariantType's are in the data set
input=open('merged_annot_filter1.vcf')

var=dict()

for line in input:
    if line.startswith('#'):
        continue
    row=line.rstrip().split('\t')
    for i in row[7].split(';'):
        if i.startswith('VariantType='):
            data=i.split('VariantType=')[1].split('.')[0]
    if data in var:
        var[data]+=1
    else:
        var[data]=1
print(var)

{'COMPLEX': 2, 'INSERTION': 117963, 'MULTIALLELIC_COMPLEX': 11061, 'MULTIALLELIC_SNP': 517, 'DELETION': 190667, 'SNP': 1465622}


In [8]:
# summary stats on INDELs and SNPs for unique variants in Line 6 mice
def SummaryStatsOfVariantType(FileName):
    '''This generator separates SNPs and INDELs in the file
    and counts up their Annotation Types based on Annovar data added to INFO'''

    input=open(FileName)

    type_snp=dict()
    type_indel=dict()

    for line in input:
        if line.startswith('#'):
            continue
        variant='COMPLEX'
        type=''
        row=line.rstrip().split('\t')
        for i in row[7].split(';'):
            if i.startswith('VariantType='):
                data=i.split('VariantType=')[1].split('.')[0]
                if data.startswith('DELETION') or data.startswith('INSERTION'):
                    variant='INDEL'
                if data.startswith('SNP') or data.startswith('MULTIALLELIC_SNP'):
                    variant='SNP'
            if i.startswith('AnnovarType='):
                type=i.split('=')[1]

        if variant=='SNP':
            if not type in type_snp:
                type_snp[type]=1
            else:
                type_snp[type]+=1
        elif variant=='INDEL':
            if not type in type_indel:
                type_indel[type]=1
            else:
                type_indel[type]+=1
        else:
            continue
            
    print('SNP data:\n')
    for i in type_snp:
        print(i,type_snp[i])
    print('\nINDEL data:\n')
    for i in type_indel:
        print(i,type_indel[i])   

In [11]:
SummaryStatsOfVariantType('merged_annot_filter1.vcf')

SNP data:

splicing 63
stoploss 21
ncRNA_splicing 27
downstream 16289
UTR5 5164
ncRNA_exonic 3788
intergenic 734219
intronic 597007
unknown 247
UTR3 25336
stopgain 87
synonymous_SNV 21130
upstream 17717
nonsynonymous_SNV 11854
ncRNA_intronic 33190

INDEL data:

nonframeshift_insertion 187
frameshift_insertion 117
stoploss 1
downstream 4211
splicing 68
UTR5 912
ncRNA_exonic 592
intergenic 144695
intronic 138566
unknown 18
UTR3 7122
stopgain 4
ncRNA_splicing 3
upstream 4426
nonframeshift_deletion 273
frameshift_deletion 227
ncRNA_intronic 7208


In [9]:
SummaryStatsOfVariantType('merged_annot_filter3.vcf')

SNP data:

splicing 2
stoploss 1
downstream 66
UTR5 6
ncRNA_exonic 19
intergenic 2849
intronic 2399
UTR3 92
stopgain 3
synonymous_SNV 27
upstream 54
nonsynonymous_SNV 66
ncRNA_intronic 149

INDEL data:

splicing 1
downstream 1
UTR5 1
intergenic 92
intronic 103
UTR3 3
upstream 3
frameshift_deletion 3
ncRNA_intronic 6


In [10]:
SummaryStatsOfVariantType('merged_annot_filter4.vcf')

SNP data:

intergenic 32
intronic 35
UTR3 3
stopgain 1
synonymous_SNV 2
nonsynonymous_SNV 6
ncRNA_intronic 2

INDEL data:

intergenic 5
intronic 6
frameshift_deletion 1
ncRNA_intronic 1
